In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_recall_fscore_support
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# neural network
class EnergyEfficiencyMLP(nn.Module):
    def __init__(self, input_dim):
        super(EnergyEfficiencyMLP, self).__init__()
        # 8 neurons with relu and 50% dropout -> binary classification
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 8),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.mlp(x)

In [7]:
def get_loaders(ablation_type):
  # create paths
  BASE_PATH = '/content/drive/MyDrive/semester 4/csci 1470: final project/ablation_datasets/'
  TRAIN_DATA_PATH = BASE_PATH + ablation_type + '/train_dataset.npz'
  VAL_DATA_PATH = BASE_PATH + ablation_type + '/val_dataset.npz'
  TEST_DATA_PATH = BASE_PATH + ablation_type + '/test_dataset.npz'

  # load data from paths
  train_data = np.load(TRAIN_DATA_PATH)
  val_data = np.load(VAL_DATA_PATH)
  test_data = np.load(TEST_DATA_PATH)

  # get numpy arrays from compressed data
  X_train, y_train = train_data['X.npy'], train_data['y.npy']
  X_val, y_val = val_data['X.npy'], val_data['y.npy']
  X_test, y_test = test_data['X.npy'], test_data['y.npy']

  print(f"Training data shape: {X_train.shape}")
  print(f"Training labels shape: {y_train.shape}")
  print(f"Validation data shape: {X_val.shape}")
  print(f"Test data shape: {X_test.shape}")

  # convert numpy arrays to PyTorch tensors
  X_train_tensor = torch.FloatTensor(X_train)
  y_train_tensor = torch.FloatTensor(y_train)
  X_val_tensor = torch.FloatTensor(X_val)
  y_val_tensor = torch.FloatTensor(y_val)
  X_test_tensor = torch.FloatTensor(X_test)
  y_test_tensor = torch.FloatTensor(y_test)

  # create datasets
  train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
  val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
  test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

  # create data loaders with batch size 16
  batch_size = 16
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=batch_size)
  test_loader = DataLoader(test_dataset, batch_size=batch_size)

  # return data and loaders
  return {"data": (X_train, y_train), "loaders": (train_loader, val_loader, test_loader)}

In [6]:
def train_model(y_train, train_loader, val_loader, test_loader, input_dim, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # initialize model
    model = EnergyEfficiencyMLP(input_dim).to(device)

    # calculate class weights - make sure each class are equally important
    num_efficient = np.sum(y_train == 1)
    num_inefficient = np.sum(y_train == 0)
    total = num_efficient + num_inefficient

    weight_efficient = total / (2 * num_efficient)
    weight_inefficient = total / (2 * num_inefficient)

    print(f"distribution: efficient={num_efficient}, inefficient={num_inefficient}")
    print(f"weights: efficient={weight_efficient:.4f}, inefficient={weight_inefficient:.4f}")

    # adam optimizer with learning rate 0.0001
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    # training loop
    num_epochs = 30
    best_val_f1 = 0
    best_model = None

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # forward pass
            outputs = model(inputs).squeeze()

            # calculate loss with class weights
            weights = torch.where(labels == 1,
                               torch.tensor(weight_efficient, device=device),
                               torch.tensor(weight_inefficient, device=device))

            criterion = nn.BCELoss(weight=weights)
            loss = criterion(outputs, labels)

            # backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

        train_loss = train_loss / len(train_loader.dataset)

        # evaluate on validation set
        val_loss, val_metrics = evaluate_model(model, val_loader, device)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")
        print(f"Val Metrics: Precision={val_metrics['precision']:.4f}, Recall={val_metrics['recall']:.4f}, F1={val_metrics['f1']:.4f}")

        # save best model based on validation F1 score
        if val_metrics['f1'] > best_val_f1:
            best_val_f1 = val_metrics['f1']
            best_model = model.state_dict().copy()

    # load best model
    model.load_state_dict(best_model)

    # evaluation on test set
    test_loss, test_metrics = evaluate_model(model, test_loader, device)

    print("\nTest Results:")
    print(f"Loss: {test_loss:.4f}")
    print(f"Precision: {test_metrics['precision']:.4f}")
    print(f"Recall: {test_metrics['recall']:.4f}")
    print(f"F1 Score: {test_metrics['f1']:.4f}")

    return model, test_metrics


def evaluate_model(model, data_loader, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # forward pass + remove dim of 1
            outputs = model(inputs).squeeze()

            # binary cross entrpy
            criterion = nn.BCELoss()
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)

            # convert outputs to binary predictions
            preds = (outputs > 0.5).float()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # calculate loss
    loss = running_loss / len(data_loader.dataset)

    # calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='macro', zero_division=0)

    metrics = {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

    return loss, metrics


**Train MLP Model**

In [ ]:
# input dim
input_dim = X_train.shape[1]

# train model
model, test_metrics = train_model(train_loader, val_loader, test_loader, input_dim, device)

# model path
model_path = '/content/drive/MyDrive/semester 4/csci 1470: final project/' + 'energy_efficiency_model.pth'

# save trained model
torch.save(model.state_dict(), model_path)

**Ablation Studies**

In [ ]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

ablations = ['av', 'fp', 'lst', 'sv']
for ablation in ablations:
  print(f"ablation: {ablation}")
  # get data and loaders
  data_loader_dict = get_loaders(ablation)

  train_loader, val_loader, test_loader = data_loader_dict['loaders']
  X_train, y_train = data_loader_dict['data']

  # input dim
  input_dim = X_train.shape[1]

  # train model
  model, test_metrics = train_model(y_train, train_loader, val_loader, test_loader, input_dim, device)

  # model path
  model_path = '/content/drive/MyDrive/semester 4/csci 1470: final project/ablation_models/' + ablation + '_energy_efficiency_model.pth'
  print(f"path: {model_path}")

  # save trained model
  torch.save(model.state_dict(), model_path)
  print(f"saved!\n\n")

Using device: cpu
ablation: av
Training data shape: (27922, 2048)
Training labels shape: (27922,)
Validation data shape: (2805, 2048)
Test data shape: (2756, 2048)
distribution: efficient=18198, inefficient=9724
weights: efficient=0.7672, inefficient=1.4357
Epoch 1/30 - Train Loss: 0.6837 - Val Loss: 0.6777
Val Metrics: Precision=0.5861, Recall=0.5944, F1=0.5683
Epoch 2/30 - Train Loss: 0.6773 - Val Loss: 0.6711
Val Metrics: Precision=0.5882, Recall=0.5972, F1=0.5755
Epoch 3/30 - Train Loss: 0.6747 - Val Loss: 0.6782
Val Metrics: Precision=0.5943, Recall=0.6022, F1=0.5696
Epoch 4/30 - Train Loss: 0.6727 - Val Loss: 0.6578
Val Metrics: Precision=0.5871, Recall=0.5952, F1=0.5843
Epoch 5/30 - Train Loss: 0.6711 - Val Loss: 0.6566
Val Metrics: Precision=0.5867, Recall=0.5929, F1=0.5868
Epoch 6/30 - Train Loss: 0.6719 - Val Loss: 0.6680
Val Metrics: Precision=0.5875, Recall=0.5961, F1=0.5710
Epoch 7/30 - Train Loss: 0.6692 - Val Loss: 0.6825
Val Metrics: Precision=0.5902, Recall=0.5966, F1=

In [8]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

ablations = ['av_fp', 'av_lst', 'av_sv', 'sv_fp', 'sv_lst', 'lst_fp']
for ablation in ablations:
  print(f"ablation: {ablation}")
  # get data and loaders
  data_loader_dict = get_loaders(ablation)

  train_loader, val_loader, test_loader = data_loader_dict['loaders']
  X_train, y_train = data_loader_dict['data']

  # input dim
  input_dim = X_train.shape[1]

  # train model
  model, test_metrics = train_model(y_train, train_loader, val_loader, test_loader, input_dim, device)

  # model path
  model_path = '/content/drive/MyDrive/semester 4/csci 1470: final project/ablation_models/' + ablation + '_energy_efficiency_model.pth'
  print(f"path: {model_path}")

  # save trained model
  torch.save(model.state_dict(), model_path)
  print(f"saved!\n\n")

Using device: cpu
ablation: av_fp
Training data shape: (27922, 2049)
Training labels shape: (27922,)
Validation data shape: (2805, 2049)
Test data shape: (2756, 2049)
distribution: efficient=18198, inefficient=9724
weights: efficient=0.7672, inefficient=1.4357
Epoch 1/30 - Train Loss: 0.6892 - Val Loss: 0.7272
Val Metrics: Precision=0.6137, Recall=0.5112, F1=0.2896
Epoch 2/30 - Train Loss: 0.6818 - Val Loss: 0.7071
Val Metrics: Precision=0.6053, Recall=0.5600, F1=0.4263
Epoch 3/30 - Train Loss: 0.6802 - Val Loss: 0.6991
Val Metrics: Precision=0.6001, Recall=0.5751, F1=0.4727
Epoch 4/30 - Train Loss: 0.6773 - Val Loss: 0.6986
Val Metrics: Precision=0.5978, Recall=0.5774, F1=0.4826
Epoch 5/30 - Train Loss: 0.6770 - Val Loss: 0.6903
Val Metrics: Precision=0.5991, Recall=0.5918, F1=0.5208
Epoch 6/30 - Train Loss: 0.6761 - Val Loss: 0.6990
Val Metrics: Precision=0.5967, Recall=0.5683, F1=0.4585
Epoch 7/30 - Train Loss: 0.6745 - Val Loss: 0.6795
Val Metrics: Precision=0.5993, Recall=0.6046, 

In [9]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

ablations = ['av_lst_fp', 'av_sv_lst', 'av_sv_fp', 'sv_lst_fp']
for ablation in ablations:
  print(f"ablation: {ablation}")
  # get data and loaders
  data_loader_dict = get_loaders(ablation)

  train_loader, val_loader, test_loader = data_loader_dict['loaders']
  X_train, y_train = data_loader_dict['data']

  # input dim
  input_dim = X_train.shape[1]

  # train model
  model, test_metrics = train_model(y_train, train_loader, val_loader, test_loader, input_dim, device)

  # model path
  model_path = '/content/drive/MyDrive/semester 4/csci 1470: final project/ablation_models/' + ablation + '_energy_efficiency_model.pth'
  print(f"path: {model_path}")

  # save trained model
  torch.save(model.state_dict(), model_path)
  print(f"saved!\n\n")

Using device: cpu
ablation: av_lst_fp
Training data shape: (27922, 2052)
Training labels shape: (27922,)
Validation data shape: (2805, 2052)
Test data shape: (2756, 2052)
distribution: efficient=18198, inefficient=9724
weights: efficient=0.7672, inefficient=1.4357
Epoch 1/30 - Train Loss: 0.6962 - Val Loss: 0.6797
Val Metrics: Precision=0.3264, Recall=0.5000, F1=0.3950
Epoch 2/30 - Train Loss: 0.6939 - Val Loss: 0.6833
Val Metrics: Precision=0.3264, Recall=0.5000, F1=0.3950
Epoch 3/30 - Train Loss: 0.6903 - Val Loss: 0.6862
Val Metrics: Precision=0.3264, Recall=0.5000, F1=0.3950
Epoch 4/30 - Train Loss: 0.6859 - Val Loss: 0.6875
Val Metrics: Precision=0.5940, Recall=0.5584, F1=0.4342
Epoch 5/30 - Train Loss: 0.6855 - Val Loss: 0.6910
Val Metrics: Precision=0.5985, Recall=0.5546, F1=0.4184
Epoch 6/30 - Train Loss: 0.6831 - Val Loss: 0.6822
Val Metrics: Precision=0.6042, Recall=0.5965, F1=0.5251
Epoch 7/30 - Train Loss: 0.6813 - Val Loss: 0.6891
Val Metrics: Precision=0.5968, Recall=0.56

**Linear Prediction Head**

In [11]:
import torch
import torch.nn as nn

class EnergyEfficiencyLinear(nn.Module):
    def __init__(self, input_dim=4100):
        super(EnergyEfficiencyLinear, self).__init__()

        # single layer -> binary classigication
        self.prediction_head = nn.Sequential(
            nn.Linear(input_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.prediction_head(x)

In [12]:
def train_linear_model(train_loader, val_loader, test_loader, input_dim, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # initialize model
    model = EnergyEfficiencyLinear(input_dim).to(device)

    # calculate class weights - make sure each class are equally important
    num_efficient = np.sum(y_train == 1)
    num_inefficient = np.sum(y_train == 0)
    total = num_efficient + num_inefficient

    weight_efficient = total / (2 * num_efficient)
    weight_inefficient = total / (2 * num_inefficient)

    print(f"distribution: efficient={num_efficient}, inefficient={num_inefficient}")
    print(f"weights: efficient={weight_efficient:.4f}, inefficient={weight_inefficient:.4f}")

    # adam optimizer with learning rate 0.0001
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    # training loop
    num_epochs = 30
    best_val_f1 = 0
    best_model = None

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # forward pass
            outputs = model(inputs).squeeze()

            # calculate loss with class weights
            weights = torch.where(labels == 1,
                               torch.tensor(weight_efficient, device=device),
                               torch.tensor(weight_inefficient, device=device))

            criterion = nn.BCELoss(weight=weights)
            loss = criterion(outputs, labels)

            # backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

        train_loss = train_loss / len(train_loader.dataset)

        # evaluate on validation set
        val_loss, val_metrics = evaluate_model(model, val_loader, device)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")
        print(f"Val Metrics: Precision={val_metrics['precision']:.4f}, Recall={val_metrics['recall']:.4f}, F1={val_metrics['f1']:.4f}")

        # save best model based on validation F1 score
        if val_metrics['f1'] > best_val_f1:
            best_val_f1 = val_metrics['f1']
            best_model = model.state_dict().copy()

    # load best model
    model.load_state_dict(best_model)

    # evaluation on test set
    test_loss, test_metrics = evaluate_linear_model(model, test_loader, device)

    print("\nTest Results:")
    print(f"Loss: {test_loss:.4f}")
    print(f"Precision: {test_metrics['precision']:.4f}")
    print(f"Recall: {test_metrics['recall']:.4f}")
    print(f"F1 Score: {test_metrics['f1']:.4f}")

    return model, test_metrics


def evaluate_linear_model(model, data_loader, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # forward pass + remove dim of 1
            outputs = model(inputs).squeeze()

            # binary cross entrpy
            criterion = nn.BCELoss()
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)

            # convert outputs to binary predictions
            preds = (outputs > 0.5).float()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # calculate loss
    loss = running_loss / len(data_loader.dataset)

    # calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='macro')

    metrics = {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

    return loss, metrics


**Train Linear Model**

In [ ]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# input dim
input_dim = X_train.shape[1]

# train model
linear_model, linear_metrics = train_model(train_loader, val_loader, test_loader, input_dim, device)

# save trained model
torch.save(model.state_dict(), '/content/drive/MyDrive/semester 4/csci 1470: final project/energy_efficiency_linear_model.pth')

Using device: cpu
distribution: efficient=18198, inefficient=9724
weights: efficient=0.7672, inefficient=1.4357
Epoch 1/30 - Train Loss: 0.6898 - Val Loss: 0.6945
Val Metrics: Precision=0.5890, Recall=0.5978, F1=0.5734
Epoch 2/30 - Train Loss: 0.6855 - Val Loss: 0.6921
Val Metrics: Precision=0.5948, Recall=0.6046, F1=0.5859
Epoch 3/30 - Train Loss: 0.6834 - Val Loss: 0.6856
Val Metrics: Precision=0.5915, Recall=0.5949, F1=0.5926
Epoch 4/30 - Train Loss: 0.6809 - Val Loss: 0.6823
Val Metrics: Precision=0.5952, Recall=0.5978, F1=0.5962
Epoch 5/30 - Train Loss: 0.6788 - Val Loss: 0.6797
Val Metrics: Precision=0.5970, Recall=0.6016, F1=0.5982
Epoch 6/30 - Train Loss: 0.6780 - Val Loss: 0.6765
Val Metrics: Precision=0.5975, Recall=0.5991, F1=0.5982
Epoch 7/30 - Train Loss: 0.6775 - Val Loss: 0.6718
Val Metrics: Precision=0.6004, Recall=0.5899, F1=0.5920
Epoch 8/30 - Train Loss: 0.6762 - Val Loss: 0.6690
Val Metrics: Precision=0.5996, Recall=0.5825, F1=0.5837
Epoch 9/30 - Train Loss: 0.6759 

**Ablation Studies**

In [14]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

ablations = ['av', 'fp', 'lst', 'sv']
for ablation in ablations:
  print(f"ablation: {ablation}")
  # get data and loaders
  data_loader_dict = get_loaders(ablation)

  train_loader, val_loader, test_loader = data_loader_dict['loaders']
  X_train, y_train = data_loader_dict['data']

  # input dim
  input_dim = X_train.shape[1]

  # train model
  model, test_metrics = train_linear_model(train_loader, val_loader, test_loader, input_dim, device)

  # model path
  model_path = '/content/drive/MyDrive/semester 4/csci 1470: final project/ablation_models/' + ablation + '_energy_efficiency_linear_model.pth'
  print(f"path: {model_path}")

  # save trained model
  torch.save(model.state_dict(), model_path)
  print(f"saved!\n\n")

Using device: cpu
ablation: av
Training data shape: (27922, 2048)
Training labels shape: (27922,)
Validation data shape: (2805, 2048)
Test data shape: (2756, 2048)
distribution: efficient=18198, inefficient=9724
weights: efficient=0.7672, inefficient=1.4357
Epoch 1/30 - Train Loss: 0.6745 - Val Loss: 0.6989
Val Metrics: Precision=0.5939, Recall=0.5965, F1=0.5474
Epoch 2/30 - Train Loss: 0.6679 - Val Loss: 0.6551
Val Metrics: Precision=0.5865, Recall=0.5934, F1=0.5859
Epoch 3/30 - Train Loss: 0.6653 - Val Loss: 0.6411
Val Metrics: Precision=0.5806, Recall=0.5794, F1=0.5799
Epoch 4/30 - Train Loss: 0.6626 - Val Loss: 0.6920
Val Metrics: Precision=0.5929, Recall=0.5974, F1=0.5536
Epoch 5/30 - Train Loss: 0.6615 - Val Loss: 0.6824
Val Metrics: Precision=0.5901, Recall=0.5972, F1=0.5630
Epoch 6/30 - Train Loss: 0.6596 - Val Loss: 0.6540
Val Metrics: Precision=0.5900, Recall=0.5980, F1=0.5879
Epoch 7/30 - Train Loss: 0.6588 - Val Loss: 0.6700
Val Metrics: Precision=0.5915, Recall=0.6007, F1=

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Training data shape: (27922, 3)
Training labels shape: (27922,)
Validation data shape: (2805, 3)
Test data shape: (2756, 3)
distribution: efficient=18198, inefficient=9724
weights: efficient=0.7672, inefficient=1.4357
Epoch 1/30 - Train Loss: 0.7063 - Val Loss: 0.7349
Val Metrics: Precision=0.1736, Recall=0.5000, F1=0.2577
Epoch 2/30 - Train Loss: 0.6987 - Val Loss: 0.7183
Val Metrics: Precision=0.1736, Recall=0.5000, F1=0.2577
Epoch 3/30 - Train Loss: 0.6950 - Val Loss: 0.7079
Val Metrics: Precision=0.1736, Recall=0.5000, F1=0.2577
Epoch 4/30 - Train Loss: 0.6933 - Val Loss: 0.7016
Val Metrics: Precision=0.1736, Recall=0.5000, F1=0.2577
Epoch 5/30 - Train Loss: 0.6924 - Val Loss: 0.6974
Val Metrics: Precision=0.5859, Recall=0.5023, F1=0.2657
Epoch 6/30 - Train Loss: 0.6919 - Val Loss: 0.6946
Val Metrics: Precision=0.5492, Recall=0.5267, F1=0.3901
Epoch 7/30 - Train Loss: 0.6915 - Val Loss: 0.6927
Val Metrics: Precision=0.5423, Recall=0.5449, F1=0.5094
Epoch 8/30 - Train Loss: 0.6912 -

In [16]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

ablations = ['av_fp', 'av_lst', 'av_sv', 'sv_fp', 'sv_lst', 'lst_fp']
for ablation in ablations:
  print(f"ablation: {ablation}")
  # get data and loaders
  data_loader_dict = get_loaders(ablation)

  train_loader, val_loader, test_loader = data_loader_dict['loaders']
  X_train, y_train = data_loader_dict['data']

  # input dim
  input_dim = X_train.shape[1]

  # train model
  model, test_metrics = train_linear_model(train_loader, val_loader, test_loader, input_dim, device)

  # model path
  model_path = '/content/drive/MyDrive/semester 4/csci 1470: final project/ablation_models/' + ablation + '_energy_efficiency_linear_model.pth'
  print(f"path: {model_path}")

  # save trained model
  torch.save(model.state_dict(), model_path)
  print(f"saved!\n\n")

Using device: cpu
ablation: av_fp
Training data shape: (27922, 2049)
Training labels shape: (27922,)
Validation data shape: (2805, 2049)
Test data shape: (2756, 2049)
distribution: efficient=18198, inefficient=9724
weights: efficient=0.7672, inefficient=1.4357
Epoch 1/30 - Train Loss: 0.6838 - Val Loss: 0.7813
Val Metrics: Precision=0.5962, Recall=0.5504, F1=0.4090
Epoch 2/30 - Train Loss: 0.6700 - Val Loss: 0.7781
Val Metrics: Precision=0.5937, Recall=0.5573, F1=0.4313
Epoch 3/30 - Train Loss: 0.6666 - Val Loss: 0.7210
Val Metrics: Precision=0.5932, Recall=0.5863, F1=0.5158
Epoch 4/30 - Train Loss: 0.6636 - Val Loss: 0.7476
Val Metrics: Precision=0.5943, Recall=0.5773, F1=0.4875
Epoch 5/30 - Train Loss: 0.6619 - Val Loss: 0.7844
Val Metrics: Precision=0.6001, Recall=0.5659, F1=0.4478
Epoch 6/30 - Train Loss: 0.6610 - Val Loss: 0.6989
Val Metrics: Precision=0.5919, Recall=0.5944, F1=0.5456
Epoch 7/30 - Train Loss: 0.6597 - Val Loss: 0.7117
Val Metrics: Precision=0.5891, Recall=0.5879, 

In [17]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

ablations = ['av_lst_fp', 'av_sv_lst', 'av_sv_fp', 'sv_lst_fp']
for ablation in ablations:
  print(f"ablation: {ablation}")
  # get data and loaders
  data_loader_dict = get_loaders(ablation)

  train_loader, val_loader, test_loader = data_loader_dict['loaders']
  X_train, y_train = data_loader_dict['data']

  # input dim
  input_dim = X_train.shape[1]

  # train model
  model, test_metrics = train_linear_model(train_loader, val_loader, test_loader, input_dim, device)

  # model path
  model_path = '/content/drive/MyDrive/semester 4/csci 1470: final project/ablation_models/' + ablation + '_energy_efficiency_linear_model.pth'
  print(f"path: {model_path}")

  # save trained model
  torch.save(model.state_dict(), model_path)
  print(f"saved!\n\n")

Using device: cpu
ablation: av_lst_fp
Training data shape: (27922, 2052)
Training labels shape: (27922,)
Validation data shape: (2805, 2052)
Test data shape: (2756, 2052)
distribution: efficient=18198, inefficient=9724
weights: efficient=0.7672, inefficient=1.4357
Epoch 1/30 - Train Loss: 0.6879 - Val Loss: 1.0184
Val Metrics: Precision=0.1736, Recall=0.5000, F1=0.2577
Epoch 2/30 - Train Loss: 0.6704 - Val Loss: 0.8622
Val Metrics: Precision=0.5909, Recall=0.5208, F1=0.3252
Epoch 3/30 - Train Loss: 0.6669 - Val Loss: 0.8843
Val Metrics: Precision=0.5790, Recall=0.5168, F1=0.3179
Epoch 4/30 - Train Loss: 0.6630 - Val Loss: 0.7845
Val Metrics: Precision=0.6028, Recall=0.5619, F1=0.4339
Epoch 5/30 - Train Loss: 0.6614 - Val Loss: 0.7823
Val Metrics: Precision=0.6028, Recall=0.5669, F1=0.4472
Epoch 6/30 - Train Loss: 0.6596 - Val Loss: 0.7505
Val Metrics: Precision=0.6009, Recall=0.5813, F1=0.4887
Epoch 7/30 - Train Loss: 0.6576 - Val Loss: 0.7430
Val Metrics: Precision=0.6042, Recall=0.58

In [ ]:
print("\nPerformance Comparison:")
print(f"Linear Head - F1: {linear_metrics['f1']:.4f}, Precision: {linear_metrics['precision']:.4f}, Recall: {linear_metrics['recall']:.4f}")
print(f"MLP Head - F1: {test_metrics['f1']:.4f}, Precision: {test_metrics['precision']:.4f}, Recall: {test_metrics['recall']:.4f}")


Performance Comparison:
Linear Head - F1: 0.6242, Precision: 0.6818, Recall: 0.6091
MLP Head - F1: 0.6694, Precision: 0.6704, Recall: 0.6684
